# Laboratorio: Métodos de búsqueda

En las clases anteriores creaste códigos para realizar búsquedas aleatorias (Simulated Annealing) y búsquedas dirigidas (Optimización Bayesiana). Estos métodos de búsqueda se utilizan para facilitar el proceso de optimización de funciones objetivos compleja y costosas de computar.

En este laboratorio usaremos el dataset de los diferentes tipos de iris, y sus longitudes y anchos de pétalos y sépalos. Utilizaremos un RandomForest para crear un modelo de clasificación y el métrico F1 para decidir cuál es el mejor modelo de acuerdo a lo que tenemos disponible.

1. Carga el dataset de Iris

In [10]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

2. Importa el archivo `Bosque.py`.

Este archivo contiene la función `RegresionBosque`, que recibe:
- X: las características independientes
- y: la variable de respuesta
- árboles: cantidad total de árboles
- profundidad de bosque: niveles de profundidad del bosque

Su salida es:
- modelo: El objeto con el modelo ajustado
- f1: El métrico que califica qué tan bueno es el modelo que se ajustó.


In [11]:
import Bosque
modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)
f1

0.9666666666666667

### Actividad 1:

Inicializa un espacio con 5 muestras en nuestro dominio de variables independientes:
- árboles: números enteros entre 5 y 50.
- profundidad: números enteros entre 2 y 10

Utiliza optimización Bayesiana para encontrar la combinación de árboles y profundidad que **maximice** el métrico F1.

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import Bosque

def f1_objective(parametros, X, y):
    n_estimators, max_depth = parametros
    modelo, f1 = Bosque.RegresionBosque(X, y, n_estimators, max_depth)
    return f1

n_estimado_values = np.linspace(5, 50, 10, dtype=int)
max_depth_values = np.linspace(2, 10, 9, dtype=int)

X_train = np.random.uniform([5, 2], [50, 10], size=(5, 2))
y_train = np.random.rand(5)

kernel = 1.0 * RBF(length_scale=1)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gp.fit(X_train, y_train)

n = 30
f1_scores = []

for i in range(n):
    X = np.array(np.meshgrid(n_estimado_values, max_depth_values)).reshape(-1, 2)
    y_pred, y_std = gp.predict(X, return_std=True)
    next_point = X[np.argmax(y_std)]
    next_value = f1_objective(next_point, X_train, y_train)
    X_train = np.vstack([X_train, next_point])
    y_train = np.append(y_train, next_value)
    gp.fit(X_train, y_train)
    f1_scores.append(-y_train.min())

plt.plot(f1_scores)
plt.xlabel('Iteración')
plt.ylabel('Mejor puntuación F1')
plt.title('Progreso de la optimización con Simulated Annealing')
plt.show()


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

### Actividad 2:

Inicializa 2 vectores con posibles valores para las variables independientes:
- árboles: números enteros entre 5 y 50
- profundidad: números enteros entre 2 y 10

Utiliza el algoritmo de Simulated Annealing que siga el siguiente orden:
- Elige un punto de partida para las variables.
- Selecciona al azar una de las dos para modificarlas.
- Selecciona un elemento al azar de la lista que contiene los posibles valores de esa variable.
- Sigue el algoritmo ($p$ y $q$) para decidir si usas esa combinación nueva o si mantienes la anterior.

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import Bosque
import random
import math

def f1_objective(parametros, X, y):
    n_estimators, max_depth = parametros
    modelo, f1 = Bosque.RegresionBosque(X, y, n_estimators, max_depth)
    return f1

n_estimado_values = np.linspace(5, 50, 10, dtype=int)
max_depth_values = np.linspace(2, 10, 9, dtype=int)

X_train = np.random.uniform([5, 2], [50, 10], size=(5, 2))
y_train = np.random.rand(5)

kernel = 1.0 * RBF(length_scale=1)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gp.fit(X_train, y_train)

current_point = [random.choice(n_estimado_values), random.choice(max_depth_values)]
current_value = f1_objective(current_point, X_train, y_train)

temp_inicial = 100
tasa_enfriamiento = 0.95
n_iteraciones = 30
f1_scores = [current_value]

for i in range(n_iteraciones):
    if random.choice([True, False]):
        new_point = [random.choice(n_estimado_values), current_point[1]]
    else:
        new_point = [current_point[0], random.choice(max_depth_values)]

    new_value = f1_objective(new_point, X_train, y_train)

    delta = new_value - current_value

    if delta > 0:
        current_point = new_point
        current_value = new_value
    else:
        probability = math.exp(delta / temp_inicial)
        if random.random() < probability:
            current_point = new_point
            current_value = new_value

    temp_inicial *= tasa_enfriamiento

    f1_scores.append(current_value)

plt.plot(f1_scores)
plt.xlabel('Iteración')
plt.ylabel('Mejor puntuación F1')
plt.title('Progreso de la optimización con Simulated Annealing')
plt.show()



ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.